# Getting Started with Multi-Modal Neural Network

This notebook provides a step-by-step guide to setting up the workspace for the multi-modal neural network project with double-loop learning and Wolfram Alpha integration.

## 1. Install Python and Pip

First, ensure Python 3.10+ and pip are installed on your system.

In [ ]:
# Check Python version
import sys
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")

# Check if pip is available
try:
    import pip
    print(f"Pip version: {pip.__version__}")
except ImportError:
    print("Pip is not installed. Please install pip.")
    
# Verify minimum Python version
version_info = sys.version_info
if version_info.major >= 3 and version_info.minor >= 10:
    print("✓ Python version is compatible (3.10+)")
else:
    print("✗ Python 3.10 or higher is required")

## 2. Create Virtual Environment

Create a virtual environment to isolate project dependencies.

In [ ]:
# Note: Virtual environment creation is typically done from the terminal
# If running this in Jupyter, the environment is already active

import os
import sys

# Check if we're in a virtual environment
in_venv = hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix)

if in_venv:
    print("✓ Running in a virtual environment")
    print(f"  Environment path: {sys.prefix}")
else:
    print("Not in a virtual environment")
    print("Tip: Create and activate a venv with:")
    print("  python -m venv .venv")
    print("  .venv\\Scripts\\activate  (Windows)")
    print("  source .venv/bin/activate  (Linux/Mac)")

## 3. Activate Virtual Environment

Activate the virtual environment.

In [ ]:
# Activate virtual environment (this may not work in notebook, do it in terminal)
# In terminal: venv\Scripts\activate (Windows) or source venv/bin/activate (Linux/Mac)

print("Please activate the virtual environment in your terminal before proceeding.")

## 4. Install Required Packages

Install the project dependencies.

In [ ]:
# Install requirements
!pip install -r ../requirements.txt

print("Dependencies installed.")

## 5. Set Up Project Directory

The project directory structure is already created. Verify it exists.

In [ ]:
# Check project structure
import os

dirs = ['src', 'configs', 'notebooks', 'tests', 'docs', 'examples']
for d in dirs:
    if os.path.exists(f'../{d}'):
        print(f"✓ {d} directory exists")
    else:
        print(f"✗ {d} directory missing")

## 6. Configure Git Repository

Initialize Git if not already done.

In [ ]:
# Check Git repository status
import os
import sys

# Ensure project root is importable when running from notebooks/
if '..' not in sys.path:
    sys.path.insert(0, '..')

from src.utils.subprocess_utils import _safe_subprocess_run

try:
    # Check if we're in a git repository
    result = _safe_subprocess_run(['git', 'rev-parse', '--git-dir'], timeout=5, cwd='..')
    if result and result.returncode == 0:
        print("✓ Git repository initialized")
        
        # Get current branch
        branch_result = _safe_subprocess_run(['git', 'branch', '--show-current'], timeout=5, cwd='..')
        if branch_result and branch_result.stdout.strip():
            print(f"  Current branch: {branch_result.stdout.strip()}")
        
        # Check for uncommitted changes
        status_result = _safe_subprocess_run(['git', 'status', '--short'], timeout=5, cwd='..')
        if status_result and status_result.stdout.strip():
            print("  Uncommitted changes detected")
        else:
            print("  Working directory clean")
    else:
        print("Git repository not initialized")
        print("Run: git init")
        
except FileNotFoundError:
    print("Git not found. Please install Git.")

# Check .gitignore
gitignore_path = '../.gitignore'
if os.path.exists(gitignore_path):
    print("✓ .gitignore exists")
else:
    print("✗ .gitignore not found")

## 7. Run Initial Tests

Run a simple test to verify the setup.

In [ ]:
# Verify key dependencies
import sys
sys.path.append('..')

try:
    import torch
    print(f"✓ PyTorch version: {torch.__version__}")
    print(f"  CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"  CUDA version: {torch.version.cuda}")
        print(f"  GPU: {torch.cuda.get_device_name(0)}")
except ImportError:
    print("✗ PyTorch not installed")

try:
    import transformers
    print(f"✓ Transformers version: {transformers.__version__}")
except ImportError:
    print("✗ Transformers not installed")

try:
    from src.training.trainer import Trainer
    from src.models.multi_modal_model import MultiModalModel
    from src.data.dataset import MultiModalDataset
    print("✓ Project modules imported successfully")
except ImportError as e:
    print(f"✗ Failed to import project modules: {e}")

print("\n✓ Setup verification complete!")

## 8. GPU Detection and Configuration

Check if GPU is available and configure for training.

In [ ]:
# Detect GPU configuration
from src.utils.gpu_utils import detect_gpu_info, print_gpu_info, check_mixed_precision_support

# Get detailed GPU information
gpu_info = detect_gpu_info()
print_gpu_info(gpu_info)

# Check mixed precision support
print("\nMixed Precision Training Support:")
mp_support = check_mixed_precision_support()
print(f"  FP16 (Half Precision): {'✅ Supported' if mp_support['fp16'] else '❌ Not supported'}")
print(f"  BF16 (BFloat16): {'✅ Supported' if mp_support['bf16'] else '❌ Not supported'}")
print(f"  TF32 (TensorFloat32): {'✅ Supported' if mp_support['tf32'] else '❌ Not supported'}")

# Check for external GPUs
if gpu_info.get('external_gpu_count', 0) > 0:
    print("\n" + "="*70)
    print("EXTERNAL GPU DETECTED")
    print("="*70)
    print(f"🔌 Found {gpu_info['external_gpu_count']} external GPU(s)")
    for device in gpu_info['devices']:
        if device.get('is_external', False):
            print(f"\n  • {device['name']}")
            if device.get('connection_type'):
                print(f"    Connection: {device['connection_type']}")
            print(f"    Memory: {device['total_memory_gb']:.2f} GB")
    print("\n⚠ Note: External GPUs may have 10-25% performance reduction")
    print("  due to Thunderbolt/USB bandwidth limitations.")

# Recommendations
print("\n" + "="*70)
print("RECOMMENDATIONS FOR YOUR SYSTEM:")
print("="*70)
if gpu_info['available']:
    print("✅ GPU training is available!")
    print("\nRecommended config.yaml settings:")
    print("  hardware:")
    print("    device: 'auto'  # Will automatically use GPU")
    
    if gpu_info['device_count'] > 1:
        print(f"\n✅ Multiple GPUs detected ({gpu_info['device_count']} GPUs)")
        print("  For multi-GPU training:")
        print("    device: 'cuda:0'  # Specify GPU ID")
        print("    ddp: true          # Enable Distributed Data Parallel")
    
    # Mixed precision recommendations
    if mp_support['bf16']:
        print("\n  ✅ BF16 mixed precision recommended (best for Ampere GPUs):")
        print("    training:")
        print("      mixed_precision: 'bf16'")
    elif mp_support['fp16']:
        print("\n  ✅ FP16 mixed precision recommended:")
        print("    training:")
        print("      mixed_precision: 'fp16'")
else:
    print("⚠ No GPU detected - training will use CPU")
    print("\nTo enable GPU training:")
    print("  1. Ensure you have an NVIDIA GPU")
    print("  2. Install CUDA Toolkit: https://developer.nvidia.com/cuda-downloads")
    print("  3. Reinstall PyTorch with CUDA support:")
    print("     pip uninstall torch torchvision")
    print("     pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121")
    print("\n  For CPU-only training, set in config.yaml:")
    print("    hardware:")
    print("      device: 'cpu'")

print("="*70)

## 9. NPU Detection and Configuration

Check if a Neural Processing Unit (NPU) is available on your system.

In [ ]:
# Import NPU detection utilities
import sys
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, '..')

from src.utils.npu_utils import (
    detect_npu_info,
    print_npu_info,
    check_accelerator_availability,
    get_best_available_device
)

print("=" * 80)
print("NPU Detection")
print("=" * 80)

# Detect NPU
npu_info = detect_npu_info()
print_npu_info(npu_info)

print("\n" + "=" * 80)
print("Accelerator Availability")
print("=" * 80)

# Check all accelerators
availability = check_accelerator_availability()
print(f"CUDA (NVIDIA GPU): {'✓ Available' if availability['cuda'] else '✗ Not available'}")
print(f"MPS (Apple Silicon): {'✓ Available' if availability['mps'] else '✗ Not available'}")
print(f"NPU: {'✓ Available' if availability['npu'] else '✗ Not available'}")
print(f"CPU: {'✓ Available' if availability['cpu'] else '✗ Not available'}")

print("\n" + "=" * 80)
print("Recommended Device")
print("=" * 80)

# Get recommended device (prefer GPU by default)
recommended_device = get_best_available_device(prefer_npu=False)
print(f"Default recommendation: {recommended_device}")

# Get recommended device with NPU preference
recommended_with_npu = get_best_available_device(prefer_npu=True)
print(f"With NPU preference: {recommended_with_npu}")

print("\n" + "=" * 80)
print("Configuration Tips")
print("=" * 80)

if npu_info['available']:
    npu_location = "🔌 External" if npu_info.get('is_external', False) else "💻 Internal"
    print(f"✓ NPU Detected ({npu_location}): {npu_info['device_name']}")
    print(f"  Type: {npu_info['npu_type']}")
    if npu_info.get('is_external', False):
        print(f"  Connection: {npu_info.get('connection_type', 'Unknown')}")
        print("\n  External NPU devices (like Coral Edge TPU or Intel NCS2):")
        print("  • Best for edge inference and prototyping")
        print("  • Lower power consumption than GPUs")
        print("  • Export model to ONNX/TFLite for NPU inference")
    if npu_info['backend']:
        print(f"  Backend: {npu_info['backend']}")
    print("\nTo use NPU in training:")
    print("  1. Set hardware.device: 'npu' in configs/default.yaml")
    print("  2. Or set hardware.prefer_npu: true for automatic selection")
    print("\nNote: PyTorch NPU support is limited. Consider exporting to ONNX for NPU inference.")
else:
    print("✗ No NPU detected on this system")
    print("\nSupported NPUs:")
    print("  - Intel AI Boost (Meteor Lake and later)")
    print("  - AMD Ryzen AI (Ryzen 7040/8040 series)")
    print("  - Apple Neural Engine (M1/M2/M3)")
    print("  - Qualcomm Hexagon NPU (Windows on ARM)")

print("=" * 80)

## 10. Development Tools

Check the development tools available for the project.

In [ ]:
# Check development tools availability
import shutil
import subprocess

print("=" * 70)
print("DEVELOPMENT TOOLS STATUS")
print("=" * 70)

# Check for make
make_path = shutil.which('make')
print(f"\n{'✓' if make_path else '✗'} make: {'Available' if make_path else 'Not found'}")
if make_path:
    print("  Quick commands:")
    print("    make test      - Run tests")
    print("    make test-cov  - Run tests with coverage")
    print("    make lint      - Run linters")
    print("    make format    - Format code")

# Check for pre-commit
try:
    import pre_commit
    print(f"\n✓ pre-commit: Installed")
    print("  Run: pre-commit install (one-time setup)")
except ImportError:
    print(f"\n✗ pre-commit: Not installed")
    print("  Install: pip install pre-commit")

# Check for pytest
try:
    import pytest
    print(f"\n✓ pytest: {pytest.__version__}")
except ImportError:
    print(f"\n✗ pytest: Not installed")

# Check for ruff
ruff_path = shutil.which('ruff')
print(f"\n{'✓' if ruff_path else '✗'} ruff: {'Available' if ruff_path else 'Not found'}")

# Check for bandit
bandit_path = shutil.which('bandit')
print(f"{'✓' if bandit_path else '✗'} bandit: {'Available' if bandit_path else 'Not found'}")

# Check for mypy
mypy_path = shutil.which('mypy')
print(f"{'✓' if mypy_path else '✗'} mypy: {'Available' if mypy_path else 'Not found'}")

print("\n" + "=" * 70)
print("TESTING STATUS")
print("=" * 70)

# Run a quick test check
try:
    from src.utils.subprocess_utils import _safe_subprocess_run
    result = _safe_subprocess_run(
        ['python', '-m', 'pytest', '--collect-only', '-q', '../tests'],
        timeout=30,
        cwd='.'
    )
    if result and result.returncode == 0:
        lines = result.stdout.strip().split('\n')
        for line in lines[-5:]:  # Show last 5 lines
            if 'test' in line.lower() or 'error' in line.lower():
                print(line)
    else:
        print("Run 'make test' or 'pytest tests/' to run the test suite")
except Exception as e:
    print(f"Could not collect tests: {e}")

print("\n" + "=" * 70)
print("For full documentation, see:")
print("  - README.md")
print("  - TRAINING_GUIDE.md")
print("  - docs/GPU_TRAINING.md")
print("  - docs/NPU_TRAINING.md")
print("=" * 70)